In [59]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re

In [60]:
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

In [61]:
#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
time=[]
genre = []
rating=[]
metascore =[]
votes = []
gross = []
description = []

In [62]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

In [63]:
#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(re.findall("\d+", year_of_release)[0])
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') else np.nan
        runtime = np.nan if runtime is np.nan else runtime[:len(runtime) - 4]
        time.append(runtime)
        
        movie_genre = store.p.find("span", class_ = 'genre').text.split(",")[0].replace('\n', '') if store.p.find("span", class_ = 'genre') else np.nan
        genre.append(movie_genre)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else np.nan
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text.replace(',', '')
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 and re.match("\$(\d+?.?\d+?)M", value[1].text) else np.nan
        grosses = np.nan if grosses is np.nan else grosses[1:len(grosses) - 1]
        gross.append(grosses)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else np.nan
        description.append(description_)

In [64]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time (min)": time, "Genre": genre , "Movie Rating": rating, "Metascore": metascore, "Votes" : votes, "Gross ($M)": gross, "Description": description  })

In [65]:
movie_list.head(20)

,Movie Name,Year of Release,Watch Time (min),Genre,Movie Rating,Metascore,Votes,Gross ($M),Description
0,The Shawshank Redemption,1994,142,Drama,9.3,82,2734899,28.34,"Over the course of several years, two convicts..."
1,The Godfather,1972,175,Crime,9.2,100,1901837,134.97,The aging patriarch of an organized crime dyna...
2,The Dark Knight,2008,152,Action,9.0,84,2708063,534.86,When the menace known as the Joker wreaks havo...
3,Schindler's List,1993,195,Biography,9.0,95,1380679,96.90,"In German-occupied Poland during World War II,..."
4,The Lord of the Rings: The Return of the King,2003,201,Action,9.0,94,1881159,377.85,Gandalf and Aragorn lead the World of Men agai...
5,12 Angry Men,1957,96,Crime,9.0,97,809379,4.36,The jury in a New York City murder trial is fr...
6,The Godfather Part II,1974,202,Crime,9.0,90,1296523,57.30,The early life and career of Vito Corleone in ...
7,Pulp Fiction,1994,154,Crime,8.9,95,2101648,107.93,"The lives of two mob hitmen, a boxer, a gangst..."
8,Inception,2010,148,Action,8.8,74,2403875,292.58,A thief who steals corporate secrets through t...
9,Fight Club,1999,139,Drama,8.8,66,2177480,37.03,An insomniac office worker and a devil-may-car...


In [66]:
# #saving the data in excel format
movie_list.to_excel("Top 1000 IMDb movies.xlsx")

# #If you want to save the data in csv format
movie_list.to_csv("Top 1000 IMDb movies.csv")

In [67]:
print("Movie Name NaNs: ", movie_list['Movie Name'].isna().sum())
print("Year of Release NaNs: ",movie_list['Year of Release'].isna().sum())
print("Watch Time (min) NaNs: ",movie_list['Watch Time (min)'].isna().sum())
print("Genre NaNs: ",movie_list['Genre'].isna().sum())
print("Movie Rating NaNs: ",movie_list['Movie Rating'].isna().sum())
print("Metascore NaNs: ",movie_list['Metascore'].isna().sum())
print("Votes NaNs: ",movie_list['Votes'].isna().sum())
print("Gross ($M) NaNs: ",movie_list['Gross ($M)'].isna().sum())
print("Description NaNs: ",movie_list['Description'].isna().sum())

Movie Name NaNs:  0
Year of Release NaNs:  0
Watch Time (min) NaNs:  0
Genre NaNs:  0
Movie Rating NaNs:  0
Metascore NaNs:  156
Votes NaNs:  0
Gross ($M) NaNs:  185
Description NaNs:  0


In [68]:
#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
time=[]
genre = []
rating=[]
metascore =[]
votes = []
gross = []
description = []

In [69]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,10000,100)

In [70]:
#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(re.findall("\d+", year_of_release)[0])
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') else np.nan
        runtime = np.nan if runtime is np.nan else runtime[:len(runtime) - 4]
        time.append(runtime)
        
        movie_genre = store.p.find("span", class_ = 'genre').text.split(",")[0].replace('\n', '') if store.p.find("span", class_ = 'genre') else np.nan
        genre.append(movie_genre)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else np.nan
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text.replace(',', '')
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 and re.match("\$(\d+?.?\d+?)M", value[1].text) else np.nan
        grosses = np.nan if grosses is np.nan else grosses[1:len(grosses) - 1]
        gross.append(grosses)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else np.nan
        description.append(description_)

In [71]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time (min)": time, "Genre": genre , "Movie Rating": rating, "Metascore": metascore, "Votes" : votes, "Gross ($M)": gross, "Description": description  })

In [72]:
movie_list.head(20)

,Movie Name,Year of Release,Watch Time (min),Genre,Movie Rating,Metascore,Votes,Gross ($M),Description
0,The Shawshank Redemption,1994,142,Drama,9.3,82,2734899,28.34,"Over the course of several years, two convicts..."
1,The Godfather,1972,175,Crime,9.2,100,1901837,134.97,The aging patriarch of an organized crime dyna...
2,The Chaos Class,1975,87,Comedy,9.2,NaN,41631,NaN,"Lazy, uneducated students share a very close b..."
3,Daman,2022,121,Adventure,9.1,NaN,13199,NaN,"The film is set in 2015. Sid, is a young docto..."
4,The Dark Knight,2008,152,Action,9.0,84,2708063,534.86,When the menace known as the Joker wreaks havo...
5,Schindler's List,1993,195,Biography,9.0,95,1380679,96.90,"In German-occupied Poland during World War II,..."
6,The Lord of the Rings: The Return of the King,2003,201,Action,9.0,94,1881159,377.85,Gandalf and Aragorn lead the World of Men agai...
7,12 Angry Men,1957,96,Crime,9.0,97,809379,4.36,The jury in a New York City murder trial is fr...
8,The Godfather Part II,1974,202,Crime,9.0,90,1296523,57.30,The early life and career of Vito Corleone in ...
9,Mirror Game,2016,147,Crime,9.0,NaN,27492,NaN,Ayna is an actor and the prison is his stage. ...


In [73]:
# #saving the data in excel format
movie_list.to_excel("Top 10000 IMDb movies.xlsx")

# #If you want to save the data in csv format
movie_list.to_csv("Top 10000 IMDb movies.csv")

In [74]:
print("Movie Name NaNs: ", movie_list['Movie Name'].isna().sum())
print("Year of Release NaNs: ",movie_list['Year of Release'].isna().sum())
print("Watch Time (min) NaNs: ",movie_list['Watch Time (min)'].isna().sum())
print("Genre NaNs: ",movie_list['Genre'].isna().sum())
print("Movie Rating NaNs: ",movie_list['Movie Rating'].isna().sum())
print("Metascore NaNs: ",movie_list['Metascore'].isna().sum())
print("Votes NaNs: ",movie_list['Votes'].isna().sum())
print("Gross ($M) NaNs: ",movie_list['Gross ($M)'].isna().sum())
print("Description NaNs: ",movie_list['Description'].isna().sum())

Movie Name NaNs:  0
Year of Release NaNs:  0
Watch Time (min) NaNs:  0
Genre NaNs:  0
Movie Rating NaNs:  0
Metascore NaNs:  1999
Votes NaNs:  0
Gross ($M) NaNs:  2797
Description NaNs:  0
